# Strip PDFs and export text to csv file
- I wrote this function to automate the process of copying and pasting pdf files for a business

In [ ]:
def pdf_magic(rootdir):
    
    #TODO: update with method for extracting specific page numbers
    
    '''iterates through a folder directory, extracts text from PDFs, converts the distinct text to a dataframe,
    and exports the dataframe as a csv file with name - 'text_df' - 
     arguments: rootdir = filepath to the folder you want the function to iterate through'''
    
    text_list = []
    
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            doc = fitz.open(file)
            page = doc[0]
            text = page.getText("text")                    
                
            text_list.append(text)            
            doc.close()
            df = pd.DataFrame(text_list, columns = ['text'])
            df.to_csv('text_df.csv')                         #change name of csv file if you wish
    return df